# Hello There Fellow Novice!

Hey! i decided to do this notebook because i saw many great notebooks out there about EDA/ Feature Engineering/ Model Making, however, much of that information was really hard to understand, so thought about making it easier and better explained for people that are completely new to ML like me! 

Feedback is appreciated!

First of all, Welcome to Kaggle, this is a great place to learn about ML, and an even better place to practice.
As a newcomer myself, i may tell you one thing, don't let yourself down seeing those 100% top 1 notebooks on the top of the leaderboard, there's some shady business up there (extended datasets and stuff!!!)
Instead of aiming for the 100% which is basically out of reach with the data we're given, let's aim to improve our personal score and learn more along the way.

The Notebook below may seem confusing and scary at first, but DON'T PANIC, get your towels and come with me, slowly!

Let's get to work then!

In [1]:
from sklearn import metrics,impute

import numpy as np #for math functions
import math # for mathematical operations
import pandas as pd #for dataframe management
import lightgbm as lgb # machine learning model
import missingno as msno # for NaN visualization
import warnings # to remove the annoying warnings
import seaborn as sns # for graphical visualization

import matplotlib.pyplot as plt # for graphical visualization

from sklearn.linear_model import LinearRegression # for model making
from sklearn.model_selection import train_test_split # for model making
from sklearn.compose import make_column_transformer  # to apply Standard Scaler
from sklearn.metrics import accuracy_score # for the vizualization of the Acuracy score
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler # for data preprocessing
from sklearn.model_selection import cross_validate,GridSearchCV # to make the cross validation of the model

warnings.filterwarnings(action='ignore', category=UserWarning)

#Change size of the plot in seaborn
sns.set(rc={'figure.figsize':(11.7,8.27)})


**Let's define our datasets for training and testing:**

In [2]:
train= pd.read_csv('../input/titanic/train.csv',index_col="PassengerId")
test = pd.read_csv('../input/titanic/test.csv',index_col="PassengerId")

In [3]:
train.describe().T

**Then let's merge both to get a full dataset, which we will be working on!**

In [4]:
df_combined= pd.concat([train, test])

# **1.Study the data**
**Let's take a look at the data we are dealing with!**

**For Train Dataset**

**It seems like we have:**
12 columns,
891 rows.

**We have empty values at:**
Age,
Cabin,
Embarked.

**From the 12 columns:**
5 are integer values,
2 are float values,
5 are objects.

In [5]:
#get the information about each column of the dataset,

train.info()

**For Test Dataset**

**It seems like we have:**
11 columns,
418 rows.

**We have empty values at:**
Age,
Fare,
Cabin,
Embarked.

**From the 11 columns:**
4 are integer values,
2 are float values,
5 are objects.

In [6]:
#get the information about each column of the dataset

test.info()

**For the Combined Dataframe**

**It seems like we have:**
12 columns,
1309 rows.

**We have empty values at:**
Survived,
Age,
Fare,
Cabin,
Embarked.

**From the 12 columns:**
4 are integer values,
3 are float values,
5 are objects.

In [7]:
df_combined.info()#get the information about each column of the dataset

In [8]:
print("train columns: ", train.columns)

print("test columns: ", test.columns)

**As we can see, the Test dataset Lacks the "survived" column, which is our Target!**

In [9]:
#Select Numerical columns

num_cols = df_combined.select_dtypes(exclude="object").columns

#Select Categorical columns

cat_cols = df_combined.select_dtypes(include="object").columns

print("numerical columns are:", num_cols)
print("categorical columns are:", cat_cols)

In [10]:
#Check for unique values in each column

n_uniqe = {col:df_combined[col].nunique() for col in cat_cols}
for key in n_uniqe:
    print(f"{key} has {n_uniqe[key]} unique values")

**Seems we have duplicated and/or missing values in Ticket, and Cabin columns**

In [11]:
#get the information about each column of the dataset

train.describe()

**We can see from this description that:
our ages range from 5 months old to 80 years old,**

In [12]:
#get the information about each column of the dataset

test.describe()

In [13]:
#get the information about each column of the dataset

df_combined.describe()

In [14]:
#get the number of null values in the dataset

train.isna().sum()

In [15]:
#get the number of null values in the dataset

test.isna().sum()

In [16]:
#get the number of null values in the dataset

df_combined.isna().sum()

# **2.Exploratory Data Analysis**

**First, let's take a look at the state of our dataframe**

In [17]:
#Let's start by visualizing the empty rows on each feature using missingno

msno.matrix(df_combined)

**We have a lot of missing values on Age and Cabin, around 80% of cabin values are empty, for this model, we are going to drop this column, but feel free to try different approaches, like inputting values based on correlations!**

In [18]:
#Drop the Cabin columns
df_combined = df_combined.drop(['Cabin'], axis=1)


In [19]:
#Classify the Age group

df_combined['Age_Class'] = (df_combined['Age']/20)+0.5
df_combined['Age_Class'] = round(df_combined['Age_Class'])

#Classify the Fare group

df_combined['Fare_Class'] = (df_combined['Fare']/100)+0.5
df_combined['Fare_Class'] = round(df_combined['Fare_Class'])

**The Age group we defined is:**
* From 0 - 20 - group 1
* From 21- 40 - group 2
* From 41- 60 - group 3
* Ages 61+ - group 4 

**The Fare group we defined is:**
* From 0 - 100 - group 1
* From 101- 200 - group 2
* From 201- 300 - group 3
* From 301- 400 - group 4
* From 401- 500 - group 5
* Fares 501+ - group 6

In [20]:
#create a heatmap to identify correlation between columns

sns.heatmap(df_combined.corr(),annot=True)
plt.show()

**We have strong correlations between:**
* Fare and Survival
* Parch and SibSp(Family on board)

In [21]:
#create a bar graph with the correlation between age and survival

sns.barplot(data=df_combined,x='Age_Class',y="Survived")
plt.title("Correlation between the Age and survival")
plt.show()

We can see that younger people have greater chances of survival

In [22]:
#create a bar graph with the correlation between Age_Class and Fare

sns.barplot(data=df_combined,x='Age_Class',y="Fare")
plt.title("Correlation between the Age and Fare")
plt.show()

We can see that older people payed the higher fares

In [23]:
#create a bar graph with the correlation between Fare_Class and survival

sns.barplot(data=df_combined,x='Fare_Class',y="Survived")
plt.title("Correlation between the Fare and survival")
plt.show()

**We can see that the people who paid the highest fares had more chances of survival**

In [24]:
#Check for unique values in each column

n_uniqe = {col:df_combined[col].nunique() for col in df_combined}
for key in n_uniqe:
    print(f"{key} has {n_uniqe[key]} unique values")

**Let's get the number of relatives for each passenger by getting the sum of Parch and SibSp**

In [27]:
df_combined["SibSp"].describe().T

In [28]:
df_combined['Parch'].describe().T

In [25]:
df_combined['relatives'] = df_combined['Parch']+df_combined["SibSp"]

In [29]:
df_combined['relatives'].describe().T

**Now, let's check for the correlation between Survival and Relative count**

In [30]:
#create a bar graph with the correlation between relatives and survival

sns.barplot(data=df_combined,x='relatives',y="Survived")
plt.title("Correlation between # of relatives and survival")
plt.show()

In [31]:
#create a bar graph with the correlation between Pclass and survival - by gender

sns.factorplot(x = 'Pclass', y='Survived', 
               hue = 'Sex',data=df_combined, kind='bar')
plt.title("Correlation between the Pclass and survival")
plt.show()

**There we go! we have a major correlation between the Pclass/Sex column and survival,females at class 1 and 2 are almost certain to survive! whilst males have a way lower survival chance across all classes!**

# 3.Preprocessing Data

In [32]:
msno.matrix(df_combined)

**We still have quite a few missing values at Age, and on Age_Class consequently.
Since there aren't as much missing values as the Cabin column, let's try to fill in those Gaps using Fillna!**

In [33]:
#input the missing values in the Age and cabin column using Fillna

df_combined["Age"] = df_combined["Age"].fillna(method='ffill')


#Let's Also reset the Age_Class since we filled the columns

df_combined['Age_Class'] = (df_combined['Age']/20)+0.5
df_combined['Age_Class'] = round(df_combined['Age_Class'])

In [34]:
#Normalize the sex column setting male to 1 and female to 0

df_combined["Sex"] = [1 if x=="male" else 0 for x in df_combined["Sex"]]

In [35]:
msno.matrix(df_combined)

**Much better, we still have that one empty value at Fare and Embarked, so let's deal with that very quickly**

In [36]:
#Let's start by filling these few empty rows in Embarked and Fare

df_combined["Embarked"] = df_combined["Embarked"].fillna(method='ffill')
df_combined["Fare"] = df_combined["Fare"].fillna(method='ffill')


#Classify the Fare group again, since we filled the values

df_combined['Fare_Class'] = (df_combined['Fare']/100)+0.5
df_combined['Fare_Class'] = round(df_combined['Fare_Class'])

In [37]:
msno.matrix(df_combined)

**Now that's perfect! let's give the df a last look and see what we can do before moving on to model making!**

In [38]:
df_combined

**Seems we could encode the Ticked and Embarked values and split the names to get the last names and pronouns!**

In [39]:
# let's split the names, last names and pronouns to get better details
df_combined["Last_Name"] = [name.split(',')[0].strip() for name in df_combined["Name"]]

df_combined["Name"] = [name.split(',')[1].strip() for name in df_combined["Name"]]

df_combined["Pronoun"] = [name.split('.')[0].strip() for name in df_combined["Name"]]

df_combined["Name"] = [name.split('.')[1].strip() for name in df_combined["Name"]]

In [40]:
#let's use the label encoder to encode the categorical features

l1= LabelEncoder()

l1.fit(df_combined['Embarked'])
df_combined.Embarked = l1.transform(df_combined.Embarked)

In [41]:
l1.fit(df_combined['Ticket'])
df_combined.Ticket = l1.transform(df_combined.Ticket)

In [42]:
l1.fit(df_combined['Last_Name'])
df_combined.Last_Name = l1.transform(df_combined.Last_Name)

In [43]:
l1.fit(df_combined['Pronoun'])
df_combined.Pronoun = l1.transform(df_combined.Pronoun)

In [44]:
df_combined

**Very nice**

In [45]:
#create a heatmap to identify correlation between columns
sns.heatmap(df_combined.corr(),annot=True)
plt.show()

**Taking a Final look at the correlations between columns what can we see in the clean data is:**
* Survived and Fare have a significant correlation - endorsed by the graph we've seen earlier.
* PClass and ticked have a significant correlation however, we might want to take a closer look at it.
* Fare has significant correlations with a lot of columns - relatives, age and survival, endorsed by our previous analysis, older people tend to pay more, older people usually have relatives on board, more relatives : higher fare.

In [46]:
#let me export it and have a closer look on excel
df_combined.to_csv('Titanic_df.csv', index=False)

After looking closely, i couldn't see any major link to the ticket number, apart from the relatives which shared the ticket number

In [47]:
#let's get the clean version of the train data by dropping the empty rows in Survived

df_train = df_combined.dropna(subset=["Survived"])


#let's get the clean version of the test data by dropping the index from the train data we just got!

df_test = df_combined.drop(df_train.index)

In [48]:
#Then let's drop the target since it's not needed for the test data

df_test = df_test.drop(['Survived'], axis=1)

**Now we have a clean train dataset, let's work it out**

# 4.Model Making
For this notebook i will be using a lgb model and we will be doing parameter optimization with GridSearchCV!

In [49]:
#First, let's define our data

data = [c for c in df_train.columns if c not in ['Survived','Name']]

In [50]:
#set our data

X=df_train[data]
df_test=df_test[data]

#y is our target

y=df_train.Survived

In [51]:
#Apply train_test_split to get the train and test values for each

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [52]:
# define the parameters you want to test out in GridSearch

params={'boosting_type':['gbdt'], 
        'learning_rate':[0.8,0.5,0.3,0.1,0.01,0.001], 
        'max_depth':[1,3,5,7,9,12,15],
        'n_estimators':[100,300,500,700,1000,1500], 
        'num_leaves':[20,35,50,75,100,150,300,500], 
        'random_state':[42]}

**Don't worry if your model is taking a lot of time to fit, to get the best parameters GridSearch will run across all possible combinations of parameters within the given params so it may take a while depending on how many possibilities you put on the table!**

In [53]:
#Remember, folks we are dealing with a boolean prediction, the passenger either survived or it didn't, so we shall use a Classifier instead of a Regressor.
#I totally did not use a Regressor the first time around... O.O
# for this model i will be using LBGM due to the performance, but you can try whatever you want!

model = lgb.LGBMClassifier() 


#cv is for the number of folds for each fit, n_jobs=-1 automatically sets all the cores on the machine

#gscv= GridSearchCV(model,params,cv=5,n_jobs=-1,verbose=True)

#gscv.fit(X_train,y_train,eval_set=(X_test,y_test),early_stopping_rounds=50,verbose=False)

In [54]:
#y_preds=gscv.predict(X_train)
#t_preds=gscv.predict(X_test)
#print(accuracy_score(y_preds,y_train))
#print(accuracy_score(t_preds,y_test))

#preds = gscv.predict(df_test)
#gscv.best_params_

**After we get the best parameters, we can use those on the model and make our predictions!**

In [55]:
best_params = {'boosting_type': ['gbdt'],
 'learning_rate': [0.3],
 'max_depth': [8],
 'n_estimators': [100],
 'num_leaves': [20],
 'random_state': [42]}

In [56]:
gscv = GridSearchCV(model,best_params,cv=5,n_jobs=-1,verbose=1)
gscv.fit(X_train,y_train,eval_set=(X_test,y_test),early_stopping_rounds=5,verbose=False)

In [57]:
#let's check for overfitting!

y_preds=gscv.predict(X_train)
t_preds=gscv.predict(X_test)

print('Train accuracy: ',accuracy_score(y_preds,y_train))
print('Test accuracy: ',accuracy_score(t_preds,y_test))

preds = gscv.predict(df_test)

The Train and test scores are very close, so we can't say there is overfitting!

In [58]:
index = pd.read_csv("/kaggle/input/titanic/test.csv")["PassengerId"]
my_submission = pd.DataFrame({'PassengerId': index, "Survived": preds})

In [59]:
#remember to set the Survived values as integers, otherwise you'll get 0% accuracy

my_submission['Survived'] = my_submission['Survived'].astype('Int64')

In [61]:
my_submission.to_csv('my_submission.csv', index=False)
print('My Submission')

**You may get better results by tweaking the parameters, i encourage you to play around with the parameter optimization!**

# The End
**That's it folks, hope you learned a thing or two in this journey! be sure to upvote the notebook if you liked, and comment any constructive criticism about the model, this is more of a learning project so the aim wasn't to climb to the top of the leaderboard but to learn the concepts and the practical applications!**